# Accounting Logic

In [1]:
import random

random.seed(0)

In [2]:
global assets
global data_source
global current_prices 

assets = ['asset1', 'asset2']
data_source = (lambda assets: ({asset: random.random() for asset in assets} for i in range(1000000)))(assets)
current_prices = next(data_source)
print('current_prices: ', current_prices)

current_prices:  {'asset1': 0.8444218515250481, 'asset2': 0.7579544029403025}


In [3]:
class Portfolio:
    def __init__(self, assets: list, init_cash=1_000_000, acc_type="cash", init_margin=1.0):
        self.ledger = {asset: 0. for asset in assets}
        self.init_cash = init_cash
        self.type = acc_type
        
        self._cash = init_cash
        self.initial_margin = init_margin
        self._used_margin = 0.
        self._borrowed_margin = 0.
        self._available_margin = init_cash
        
    def __repr__(self):
        return f"""cash: {self._cash} 
                \nbalance: {self.balance}
                \nborrowed margin: {self.borrowed_margin}
                \nborrowed asset value: {self.borrowed_asset_value}
                \nequity: {self.equity} 
                \nledger: {str(self.ledger)}"""
    
    
    @property
    def cash(self):
        return self._cash 
    
    @property
    def borrowed_asset_value(self):
        return sum((current_prices[k]*v for k, v in self.ledger.items() if v < 0.))
    
    @property
    def borrowed_margin(self):
        return self._borrowed_margin
    
    @property
    def assets_value(self):
        return sum((current_prices[k]*v for k, v in self.ledger.items()))
    
    @property
    def equity(self):
        return self._cash + self.assets_value - self._borrowed_margin
        
    @property
    def balance(self):
        return self._cash 
    
    @property
    def pnl(self):
        return 
        
    @property
    def purchasing_power(self):
        return self._cash / self.initial_margin

    def transaction_cash(self, asset_name, units):
        amount = current_prices[asset_name] * units
        self._cash -= amount
        self.ledger[asset_name] += units
    
    def transaction_margin(self, asset_name, units):
        amount = current_prices[asset_name] * units
        used_margin = amount * self.initial_margin
        self._cash -= used_margin
        self._borrowed_margin += amount * (1-self.initial_margin)
        self._used_margin += used_margin
        self.ledger[asset_name] += units
        
    def order(self, asset_name: str, units: float):
        if self.type == "cash":
            self.transaction_cash(asset_name, units)
        elif self.type == "margin":
            self.transaction_margin(asset_name, units)
        return self

In [4]:
port = Portfolio(assets, acc_type="margin", init_margin=0.2)

In [5]:
port.order('asset1', 1000.), current_prices

(cash: 999831.115629695 
                 
 balance: 999831.115629695
                 
 borrowed margin: 675.5374812200386
                 
 borrowed asset value: 0
                 
 equity: 1000000.0 
                 
 ledger: {'asset1': 1000.0, 'asset2': 0.0},
 {'asset1': 0.8444218515250481, 'asset2': 0.7579544029403025})

In [6]:
def buy_test():
    cash = init_cash = 1_000_000
    port = Portfolio(assets, init_cash = init_cash, acc_type="cash")
    price = current_prices[assets[0]]
    units = 1000
    port.order(assets[0], units)
    cost = price * units
    cash -= cost
    borrowed_cash = 0.
    usedMargin = cost
    new_price = price
    equity = init_cash + units*(new_price-price)
    assert cash == port.cash
    assert borrowed_cash == port.borrowed_cash
    assert equity == port.equity

In [7]:
def transaction_test(buy=True, margin=1.):
    assert 0. <= margin <= 1., "margin (required initial) must be between 0 and 1"
    cash = init_cash = 1_000_000
    acc_type = "margin" if margin < 1. else "cash"
    port = Portfolio(assets, init_cash = init_cash, acc_type=acc_type, 
                     init_margin=margin)
    global current_prices
    num_trans=1
    borrowed_margin = 0.
    used_margin = 0.
    ledger = [0. for i in current_prices]
    for i in  range(num_trans):
        current_prices = next(data_source)
        price = current_prices[assets[0]]
        units = 1000 if buy else -1000

        # port internal accounting
        port.order(assets[0], units)

        # explicit accounting
        cost = margin*(price * units)
        cash -= cost
        borrowed_margin += (1-margin)*(price*units)
        used_margin += cost
        new_price = price
        ledger[0] += units
#         equity = init_cash + units*(new_price-price) - borrowed_margin
    equity = cash + sum(price*unit for price, unit in zip(current_prices.values(), ledger)) - borrowed_margin
#     print(cash, port.cash)
#     print(borrowed_margin, port.borrowed_margin)
#     print(equity, port.equity)
    assert cash == port.cash
    assert borrowed_margin == port.borrowed_margin
#     assert equity == init_cash, "No price change and no transaction - eq should be init_cash"
    
from functools import partial
buy_test = partial(transaction_test, buy=True)
sell_test = partial(transaction_test, buy=False)

In [8]:
ledger = [0. for i in current_prices]
ledger, current_prices

([0.0, 0.0], {'asset1': 0.8444218515250481, 'asset2': 0.7579544029403025})

In [9]:
buy_test()
buy_test(margin=0.2)
sell_test()
sell_test(margin=0.2)

## Calculate PnL without keeping track of individual positions

    Instead, keep track of average entry price for each position that isn't 0.
    When a position is incremented, the average entry price is updated by
    weighting the mean update by volume of units bought
    
    Has to be symmetric for long/short positions
    entry_price = 0. when there is no position

In [135]:
import math
sign = lambda c: math.copysign(1, c)

In [136]:
prev_price = 1.
current_units = 0.
average_entry_price = prev_price

In [168]:
current_price = 2.8
new_units = 10000

In [169]:
if sign(new_units) == -sign(current_units):
    # if new order is going the opposite direction
    # don't have to change average_entry_price if a partial close
    # If full close, current_units will be 0. and will be caught below
    if abs(new_units) > abs(current_units):
        # Reversing position
        average_entry_price = current_price
    current_units += new_units
    
else:
    # Update average entry price, weighted by the amount of units bought at the new price level
    average_entry_price += (current_price-average_entry_price) * (new_units/(new_units+current_units))    
    current_units += new_units
    
if current_units == 0:
    average_entry_price = 0.
        
print(current_units, new_units, current_price, average_entry_price)

3700.0 10000 2.8 2.8
